In [1]:
%env CUDA_VISIBLE_DEVICES=3
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '..')

import faiss
import lib
import numpy as np
from functools import partial
import torch
import os

device_ids=list(range(torch.cuda.device_count()))

env: CUDA_VISIBLE_DEVICES=3


### Description

This notebook downloads and evaluates 4 checkpoints of UNQ model trained on BIGANN1M and DEEP1M datasets with 8- and 16-byte code sizes. You can also use this code to verify the corrrectness of your setup. If all library versions & hardware are set up properly, the code below should produce the exact same outputs as you can see below. These are also the numbers we report in Table 2 of our paper. 

In [2]:
for dataset_name, checkpoint_path, link in [
    ('BIGANN1M', 'checkpoints/sift_4b/checkpoint_best.pth', 'https://www.dropbox.com/s/ycf12yqu5cw4opr/checkpoint_best.pth?dl=1'),
    ('BIGANN1M', 'checkpoints/sift_8b/checkpoint_best.pth', 'https://www.dropbox.com/s/y7aucbm5gwyow9r/checkpoint_best.pth?dl=1'),
    # ('DEEP1M', 'checkpoints/deep_8b/checkpoint_best.pth', 'https://www.dropbox.com/s/yvtm7y3f3412n9n/checkpoint_best.pth?dl=1'),
    # ('DEEP1M', 'checkpoints/deep_16b/checkpoint_best.pth', 'https://www.dropbox.com/s/a0v988tb6i00qir/checkpoint_best.pth?dl=1')
]:
    print("Evaluating checkpoint {} on dataset {}".format(checkpoint_path, dataset_name))
    
    if not os.path.exists(checkpoint_path):
        os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
        lib.utils.download(link, checkpoint_path,
                 chunk_size=4 * 1024 ** 2)
    if '2b' in checkpoint_path:
        num_codebooks = 2
    elif '4b' in checkpoint_path:
        num_codebooks = 4
    elif '8b' in checkpoint_path:
        num_codebooks = 8
    elif '16b' in checkpoint_path:
        num_codebooks = 16
    else:
        raise ValueError("Unexpected number of bytes. Make sure you know what you're doing")

    dataset = lib.Dataset(dataset_name, normalize=True)
    model = lib.UNQModel(input_dim=dataset.vector_dim, num_codebooks=num_codebooks).cuda()

    trainer = lib.Trainer(
        model=model, experiment_name='debug', device_ids=device_ids, loss_opts={},
        LearnedSimilaritySearch=partial(lib.UNQSearch, model=model, rerank_k=500, batch_size=1000,
                                        reorder_batch_size=250, device_ids=device_ids),
        NegativeSimilaritySearch=partial(lib.UNQSearch, model=model, rerank_k=1, batch_size=1000,
                                        reorder_batch_size=250, device_ids=device_ids),
    )
    trainer.load_checkpoint(path=checkpoint_path)
    print("Recall@1  :", trainer.evaluate_recall(dataset.test_vectors.cuda(), dataset.query_vectors.cuda(), dataset.gt_vectors, k=1))
    print("Recall@10 :", trainer.evaluate_recall(dataset.test_vectors.cuda(), dataset.query_vectors.cuda(), dataset.gt_vectors, k=10))
    print("Recall@100:", trainer.evaluate_recall(dataset.test_vectors.cuda(), dataset.query_vectors.cuda(), dataset.gt_vectors, k=100))

Evaluating checkpoint checkpoints/sift_4b/checkpoint_best.pth on dataset BIGANN1M
Recall@1  : 0.1001
Recall@10 : 0.3392
Recall@100: 0.7339
Evaluating checkpoint checkpoints/sift_8b/checkpoint_best.pth on dataset BIGANN1M
Recall@1  : 0.2837
Recall@10 : 0.6915
Recall@100: 0.9599
